In [12]:
import numpy as np 
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets

In [13]:
# NOTE current order of orderbook is ask1 ,bid1, ask2, bid2, which performs better than the ragular form.
# 股票预测很容易被local minimum卡住，需要更小的学习率

In [14]:
# X shape = ([price, volume], N, time_span, level)
# y shape = (N) binary label
i      = 5
stock  = 600030
Xtrain = np.load(f'dataset/{stock}-{i}.train.X.npy')
Ytrain = np.load(f'dataset/{stock}-{i}.train.y.npy')
Xtest  = np.load(f'dataset/{stock}-{i}.test.X.npy')
Ytest  = np.load(f'dataset/{stock}-{i}.test.y.npy')
Xtrain = np.log10(Xtrain + 1)
Xtest  = np.log10(Xtest + 1)
Xtrain.shape, Ytrain.shape, Xtest.shape, Ytest.shape

((2, 54890, 20, 20), (54890,), (2, 13908, 20, 20), (13908,))

In [15]:
Xtrain = torch.from_numpy(Xtrain).float().permute(1, 0, 2, 3)
Ytrain = torch.from_numpy(Ytrain).long()
Xtest  = torch.from_numpy(Xtest).float().permute(1, 0, 2, 3)
Ytest  = torch.from_numpy(Ytest).long()


#Xtrain.shape, Ytrain.shape, Xtest.shape, Ytest.shape
Xtrain[:,0].mean(), Xtrain[:,0].std(), Xtrain[:,1].mean(), Xtrain[:,1].std()

(tensor(1.2160), tensor(0.2207), tensor(5.0854), tensor(0.9532))

In [16]:
trainset = torch.utils.data.TensorDataset(Xtrain, Ytrain)
testset = torch.utils.data.TensorDataset(Xtest, Ytest)

# Create data loaders.
batch_size = 128
train_dataloader = DataLoader(trainset, batch_size=batch_size)
test_dataloader = DataLoader(testset, batch_size=batch_size)

In [17]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.convlayer = nn.Sequential(
            nn.Conv2d(2, 10, 3, 1, 1),
            nn.ReLU(),
            nn.BatchNorm2d(10),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(10, 20, 5, 1, 2),
            nn.ReLU(),
            nn.BatchNorm2d(20)
        )
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(2000, 64),
            nn.ReLU(),
            nn.Linear(64, 2),
            nn.ReLU(),
        )

    def forward(self, x):
        x = self.convlayer(x)
        x = self.flatten(x)
        return self.linear_relu_stack(x)

In [18]:
# device = "cuda" if torch.cuda.is_available() else "cpu"
device = 'cpu'
print(f"Using {device} device")

torch.manual_seed(0)
model = CNN().to(device)
print(model)

Using cpu device
CNN(
  (convlayer): Sequential(
    (0): Conv2d(2, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): ReLU()
    (6): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=2000, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=2, bias=True)
    (3): ReLU()
  )
)


In [19]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [20]:
def test(dataloader, model):
    n = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= n
    correct /= n
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [21]:
epochs = 100
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=5e-5)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    print("train ", end ='')
    test(train_dataloader, model)
    test(test_dataloader, model)
print("Done!")

Epoch 1
-------------------------------
loss: 0.573915  [    0/54890]
loss: 0.734904  [12800/54890]
loss: 0.679958  [25600/54890]
loss: 0.693262  [38400/54890]
loss: 0.688887  [51200/54890]
train Test Error: 
 Accuracy: 52.8%, Avg loss: 0.005279 

Test Error: 
 Accuracy: 56.1%, Avg loss: 0.005198 

Epoch 2
-------------------------------
loss: 0.632641  [    0/54890]
loss: 0.751540  [12800/54890]
loss: 0.667265  [25600/54890]
loss: 0.691156  [38400/54890]
loss: 0.685336  [51200/54890]
train Test Error: 
 Accuracy: 56.9%, Avg loss: 0.005192 

Test Error: 
 Accuracy: 61.1%, Avg loss: 0.005089 

Epoch 3
-------------------------------
loss: 0.813989  [    0/54890]
loss: 0.731143  [12800/54890]
loss: 0.664416  [25600/54890]
loss: 0.689401  [38400/54890]
loss: 0.684720  [51200/54890]
train Test Error: 
 Accuracy: 57.6%, Avg loss: 0.005177 

Test Error: 
 Accuracy: 61.6%, Avg loss: 0.005082 

Epoch 4
-------------------------------
loss: 0.810627  [    0/54890]
loss: 0.715117  [12800/54890]


KeyboardInterrupt: 